In [2]:
import pandas as pd
import numpy as np
import tqdm
import os
import gc

import cv2
import matplotlib.pyplot as plt
from PIL import Image

import torch
from ultralytics import YOLO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
file_path = "..\\CUB_200_2011"

images = pd.read_csv(f"{file_path}\\images.txt", 
                     sep=' ', 
                     names=['image_id', 'image_name'])
train_test = pd.read_csv(f"{file_path}\\train_test_split.txt", 
                     sep=' ', 
                     names=['image_id', 'is_training_image'])
classes = pd.read_csv(f"{file_path}\\classes.txt", 
                     sep=' ', 
                     names=['class_id', 'class_name'])
image_class_labels = pd.read_csv(f"{file_path}\\image_class_labels.txt", 
                     sep=' ', 
                     names=['image_id', 'class_id'])

bounding_boxes = pd.read_csv(f'{file_path}/bounding_boxes.txt',
                             sep=' ',
                             names=['image_id', 'x', 'y', 'width', 'height'])
bounding_boxes = bounding_boxes.astype(int)
#Merging the information given
image_data = images.merge(train_test, on='image_id', how='inner')

labels_data = classes.merge(image_class_labels, on='class_id', how='inner')

image_data = image_data.merge(labels_data, on='image_id', how='inner')

image_data = image_data.merge(bounding_boxes, on='image_id', how='inner')

image_data['image_name'] = file_path.replace('\\', '/') + \
                            '/images/' + image_data['image_name']
image_data

,image_id,image_name,is_training_image,class_id,class_name,x,y,width,height
0,1,../CUB_200_2011/images/001.Black_footed_Albatr...,0,1,001.Black_footed_Albatross,60,27,325,304
1,2,../CUB_200_2011/images/001.Black_footed_Albatr...,1,1,001.Black_footed_Albatross,139,30,153,264
2,3,../CUB_200_2011/images/001.Black_footed_Albatr...,0,1,001.Black_footed_Albatross,14,112,388,186
3,4,../CUB_200_2011/images/001.Black_footed_Albatr...,1,1,001.Black_footed_Albatross,112,90,255,242
4,5,../CUB_200_2011/images/001.Black_footed_Albatr...,1,1,001.Black_footed_Albatross,70,50,134,303
...,...,...,...,...,...,...,...,...,...
11783,11784,../CUB_200_2011/images/200.Common_Yellowthroat...,1,200,200.Common_Yellowthroat,89,95,354,250
11784,11785,../CUB_200_2011/images/200.Common_Yellowthroat...,0,200,200.Common_Yellowthroat,157,62,184,219
11785,11786,../CUB_200_2011/images/200.Common_Yellowthroat...,0,200,200.Common_Yellowthroat,190,102,198,202
11786,11787,../CUB_200_2011/images/200.Common_Yellowthroat...,1,200,200.Common_Yellowthroat,3,20,408,307


Train: `CUB_200/Train/{images, labels}`

Validation: `CUB_200/Validation/{images, labels}`

Test: `CUB_200/Test/{images, labels}`

In [3]:
# Set the first 24 training values in each group to 0 
grouped = image_data[image_data['is_training_image'] == 1].groupby('class_name')
validation_indices = []

for name, group in grouped: 
    validation_indices.extend(group.index[24:])

# Set the first 24 values in each group to 0
image_data['is_validation_image'] = 0
image_data.loc[validation_indices, 'is_validation_image'] = 1
image_data['is_validation_image'].value_counts()

is_validation_image
0    10594
1     1194
Name: count, dtype: int64

In [11]:
os.path.basename(image_data['image_name'][0])

'Black_Footed_Albatross_0046_18.jpg'

In [3]:
directory = "CUB_200_classification/"

In [8]:
if not os.path.exists(directory):
    os.makedirs(directory)

for file in ['Train', 'Validation', 'Test']:
    if not os.path.exists(f'{directory}/{file}'):
        os.makedirs(f'{directory}/{file}')
    
    for class_name in pd.unique(image_data['class_name']):
        os.makedirs(f'{directory}/{file}/{class_name}')

In [15]:
#Split the data into training and test
train_indices = (image_data['is_training_image'] == 1) & (image_data['is_validation_image'] == 0)
training_data = image_data[train_indices].reset_index()
validation_data = image_data[image_data['is_validation_image'] == 1].reset_index()
test_data = image_data[image_data['is_training_image'] == 0].reset_index()

#Setting three sets of images: training, validation, and test
X_temp_train = []
X_temp_validation = []
X_temp_test = []
temp = [X_temp_train, X_temp_validation, X_temp_test]

which_set = ['Train', 'Validation', 'Test']

for p, dataframe in enumerate([training_data, validation_data, test_data]):
    #Get the paths for the images
    paths = np.array(dataframe['image_name'])
    
    #Get the image arrays
    for i, img in tqdm.tqdm(enumerate(paths), total=dataframe.shape[0]):
        image = cv2.imread(img)
        x, y, width, height = dataframe.loc[i, ['x', 'y', 'width', 'height']]

        image = image[y:y+height, x:x+width]
        image = cv2.resize(image, (224, 224))
        cv2.imwrite(f'{directory}/{which_set[p]}/{dataframe.loc[i, "class_name"]}/{os.path.basename(img)}',
                    image)

  0%|          | 0/4800 [00:00<?, ?it/s]

100%|██████████| 5794/5794 [01:22<00:00, 70.16it/s]


In [9]:
os.getcwd()

'c:\\UMD\\Sem 2\\MSML605 Computing Systems\\Homeworks\\Project\\YOLOv8'

In [16]:
directory

'CUB_200_classification/'

In [17]:

# Load a model
# model = YOLO("yolov8n-cls.yaml")  # build a new model from YAML
model = YOLO("yolov8n-cls.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolov8n-cls.yaml").load("yolov8n-cls.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data=directory, epochs=100, imgsz=224)

100%|██████████| 5.31M/5.31M [00:00<00:00, 23.9MB/s]


New https://pypi.org/project/ultralytics/8.2.51 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=CUB_200_classification/, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

c:\UMD\.venv\Lib\site-packages\torch\nn\modules\module.py:1159: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


AMP: checks passed 


train: Scanning C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... 4800 images, 0 corrupt: 100%|██████████| 4800/4800 [00:04<00:00, 1002.75it/s]


...

    100/100     0.247G    0.05678         16        224: 100%|██████████| 300/300 [00:13<00:00, 21.72it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 56.38it/s]

                   all       0.73      0.915



100 epochs completed in 0.437 hours.
Optimizer stripped from runs\classify\train25\weights\last.pt, 3.5MB
Optimizer stripped from runs\classify\train25\weights\best.pt, 3.5MB

Validating runs\classify\train25\weights\best.pt...
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLOv8n-cls summary (fused): 73 layers, 1691080 parameters, 0 gradients, 3.5 GFLOPs
train: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... found 4800 images in 200 classes  
val: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\validation... found 1194 images in 200 classes  
test: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\test... found 5794 images in 200 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 42.84it/s]


                   all      0.731       0.92
Speed: 0.3ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train25
Results saved to runs\classify\train25


In [6]:

# Load a model
# model = YOLO("yolov8n-cls.yaml")  # build a new model from YAML
model_s = YOLO("yolov8s-cls.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolov8n-cls.yaml").load("yolov8n-cls.pt")  # build from YAML and transfer weights

# Train the model
results_s = model_s.train(data=directory, epochs=100, imgsz=224)

New https://pypi.org/project/ultralytics/8.2.51 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=CUB_200_classification/, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... 4800 images, 0 corrupt: 100%|██████████| 4800/4800 [00:00<?, ?it/s]
val: Scanning C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\validation... 1194 images, 0 corrupt: 100%|██████████| 1194/1194 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train28
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      0.86G      5.125         16        224: 100%|██████████| 300/300 [00:11<00:00, 26.52it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 90.54it/s]

                   all      0.168      0.449


...

    100/100     0.489G    0.02633         16        224: 100%|██████████| 300/300 [00:07<00:00, 39.49it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 97.31it/s] 

                   all      0.774      0.934



100 epochs completed in 0.283 hours.
Optimizer stripped from runs\classify\train28\weights\last.pt, 10.8MB
Optimizer stripped from runs\classify\train28\weights\best.pt, 10.8MB

Validating runs\classify\train28\weights\best.pt...
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLOv8s-cls summary (fused): 73 layers, 5331400 parameters, 0 gradients, 12.7 GFLOPs
train: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... found 4800 images in 200 classes  
val: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\validation... found 1194 images in 200 classes  
test: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\test... found 5794 images in 200 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 84.32it/s]


                   all      0.774      0.936
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train28
Results saved to runs\classify\train28


In [5]:
# Load a model
# model = YOLO("yolov8n-cls.yaml")  # build a new model from YAML
model_s = YOLO("yolov8m-cls.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolov8n-cls.yaml").load("yolov8n-cls.pt")  # build from YAML and transfer weights

# Train the model
results_s = model_s.train(data=directory, epochs=100, imgsz=224)

100%|██████████| 32.7M/32.7M [00:01<00:00, 21.1MB/s]


New https://pypi.org/project/ultralytics/8.2.51 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=CUB_200_classification/, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... 4800 images, 0 corrupt: 100%|██████████| 4800/4800 [00:00<?, ?it/s]
val: Scanning C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\validation... 1194 images, 0 corrupt: 100%|██████████| 1194/1194 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train27
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      1.03G      5.085         16        224: 100%|██████████| 300/300 [00:16<00:00, 17.86it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 51.52it/s]

...

    100/100     0.944G    0.01937         16        224: 100%|██████████| 300/300 [00:12<00:00, 24.88it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 63.20it/s]

                   all      0.778      0.937



100 epochs completed in 0.498 hours.
Optimizer stripped from runs\classify\train27\weights\last.pt, 32.2MB
Optimizer stripped from runs\classify\train27\weights\best.pt, 32.2MB

Validating runs\classify\train27\weights\best.pt...
Ultralytics YOLOv8.2.38  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLOv8m-cls summary (fused): 103 layers, 16018856 parameters, 0 gradients, 41.8 GFLOPs
train: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\train... found 4800 images in 200 classes  
val: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\validation... found 1194 images in 200 classes  
test: C:\UMD\Sem 2\MSML605 Computing Systems\Homeworks\Project\YOLOv8\CUB_200_classification\test... found 5794 images in 200 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:00<00:00, 55.27it/s]


                   all      0.785      0.946
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train27
Results saved to runs\classify\train27
